In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from attn_gym import visualize_attention_scores
from torch.nn.attention.flex_attention import flex_attention, create_block_mask
from torch.nn.attention.flex_attention import _mask_mod_signature
from torch.nn.attention.flex_attention import (
    _score_mod_signature,
    _mask_mod_signature,
    _vmap_for_bhqkv,
    _ModificationType,
)
from torch.nn.utils.rnn import pad_sequence
from typing import List, Union
from torch import Tensor
import torch
import random

def causal_mask(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx

def _offsets_to_doc_ids_tensor(offsets):
    device = offsets.device
    offsets = offsets[offsets != -1]
    counts = offsets[1:] - offsets[:-1]
    return torch.repeat_interleave(
        torch.arange(len(counts), device=device, dtype=torch.int32), counts
    )


def length_to_offsets(lengths: List[int], device: Union[str, torch.device]) -> Tensor:
    """Converts a list of lengths to a list of offsets.

    Args:
        lengths: A list of lengths.

    """
    offsets = [0]
    offsets.extend(lengths)
    offsets = torch.tensor(offsets, device=device, dtype=torch.int32)
    offsets = torch.cumsum(offsets, dim=-1)
    return offsets

def generate_doc_mask_mod(offsets):
    
    offsets = pad_sequence(offsets, batch_first = True, padding_value = -1)
    document_id = [_offsets_to_doc_ids_tensor(offsets[i]) for i in range(offsets.shape[0])]
    document_id = torch.stack(document_id, 0)
    
    def doc_mask_mod(b, h, q_idx, kv_idx):
        same_doc = document_id[b, q_idx] == document_id[b, kv_idx]
        q_logical = q_idx - offsets[b, document_id[b, q_idx]]
        kv_logical = kv_idx - offsets[b, document_id[b, kv_idx]]
        inner_mask = causal_mask(b, h, q_logical, kv_logical)
        return same_doc & inner_mask

    return doc_mask_mod

In [3]:
flex_attention = torch.compile(flex_attention, dynamic = False)
create_block_mask = torch.compile(create_block_mask, dynamic = False)

In [4]:
device = 'cuda'

In [5]:
def generate_random_lengths(total_length, num_documents):
    # Initialize all lengths to 1 to ensure each document has at least one token
    lengths = [1] * num_documents
    remaining_length = total_length - num_documents

    # Randomly distribute the remaining length
    for _ in range(remaining_length):
        index = random.randint(0, num_documents - 1)
        lengths[index] += 1

    return lengths

max_seq_len, doc_count = 21, 4
B, H, SEQ_LEN, HEAD_DIM = 2, 1, max_seq_len, 128

offsets = []
for i in range(doc_count, doc_count + B, 1):
    lengths = generate_random_lengths(max_seq_len, i)

    offsets.append(length_to_offsets(lengths, device))

def make_tensor():
    return torch.randn(B, H, SEQ_LEN, HEAD_DIM, device=device)

q, k, v = make_tensor(), make_tensor(), make_tensor()

In [6]:
from functools import lru_cache

In [7]:
def forward(q, k, v, offsets):
    document_causal_mask = generate_doc_mask_mod(offsets)
    block_mask = create_block_mask(document_causal_mask, None, None, SEQ_LEN, SEQ_LEN, device, _compile = True)
    flex = flex_attention(q, k, v, block_mask=block_mask)
    return flex

In [8]:
%%time

flex = forward(q, k, v, offsets)

CPU times: user 2.22 s, sys: 136 ms, total: 2.36 s
Wall time: 2.53 s


In [9]:
offsets = []
extra = 7
for i in range(doc_count, doc_count + B + extra, extra):
    lengths = generate_random_lengths(max_seq_len, i)

    offsets.append(length_to_offsets(lengths, device))
offsets

[tensor([ 0,  3,  9, 14, 21], device='cuda:0'),
 tensor([ 0,  1,  2,  4,  7,  9, 11, 12, 14, 17, 20, 21], device='cuda:0')]

In [10]:
%%time

flex = forward(q, k, v, offsets)

CPU times: user 1.25 s, sys: 46.4 ms, total: 1.3 s
Wall time: 10.4 s


In [11]:
offsets = []
extra = 4
for i in range(doc_count, doc_count + B + extra, extra):
    lengths = generate_random_lengths(max_seq_len, i)

    offsets.append(length_to_offsets(lengths, device))
offsets

[tensor([ 0,  8, 15, 17, 21], device='cuda:0'),
 tensor([ 0,  1,  3,  5, 10, 13, 16, 17, 21], device='cuda:0')]

In [12]:
%%time

flex = forward(q, k, v, offsets)

CPU times: user 1.1 s, sys: 18.4 ms, total: 1.12 s
Wall time: 1.2 s
